In [ ]:
import skimage
import skimage.io
import skimage.transform

import os
import scipy as scp
import scipy.misc

import numpy as np
import logging
import sys
import threading

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO,
                    stream=sys.stdout)



In [ ]:
UNDISTORT_DATASET = False

In [ ]:
img1 = skimage.io.imread("./trees.png")
plt.figure(1)
plt.cla()
plt.imshow(img1)
print("Click desired cut position")
#click = plt.ginput()
#plt.close('all')


In [ ]:
my_dpi = 72
center = (650,358)
radius = 642.0
zero_pixel = img1[0,0]
img1.shape

In [ ]:
def polar_to_xy(r, theta):
    x = r * np.sin(theta)
    y = r * np.cos(theta)
    return x,y 
def xy_to_idx(x,y, center=(650,358), radius = 642):
    j = x * radius + center[0]
    i = y * radius + center[1]
    j = np.round(j).astype(int)
    i = np.round(i).astype(int)
    return i,j

In [ ]:
indices = np.array([(i,j) for j in range(img1.shape[1]) for i in range(img1.shape[0])])
ii = np.array([[i for j in range(img1.shape[1])] for i in range(img1.shape[0])])
jj = np.array([[j for j in range(img1.shape[1])] for i in range(img1.shape[0])])

## Square radius method (deprecated)

In [ ]:
# square radius method. Move points radially from centric shape to square shape. 
# Based on polar coordinate, new coordinate preserve polar angle, every concentric circle in original is mapped to a rounded rectangle.
# These rounded rectangles progress linearly from pure circles in the center to pure rectangles at the edges.
xx = (jj - center[0]) / radius
yy = (ii - center[1]) / radius
rr = np.sqrt(xx * xx + yy * yy)
sr = np.maximum(np.abs(xx/np.max(xx)), np.abs(yy/np.max(yy)))
sr = rr * sr + (1-rr) * rr
th = np.arctan2(xx, yy)

In [ ]:
plt.figure("circle")
plt.cla()
img2 = img1
img2[rr > 1] = 255
plt.imshow(img2)
plt.axvline(center[0], color='k')
plt.axhline(center[1], color='k')
plt.show()

In [ ]:
plt.contour(xx,yy,th,100,label="angle")
plt.contour(xx,yy,sr,100,label="square radius")
plt.show()

## Sphere Projection Method

In [ ]:
# Maps the image data to a spherical surface. Then projects points on the surface to points on a plane.

d = 0.5 # distance between center of sphere and flat surface
width = height = 2.0

In [ ]:
# Visualize spherical surface

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

#theta inclination angle
#phi azimuthal angle
n_theta = 100 # number of values for theta
n_phi = 400  # number of values for phi
r = 1        #radius of sphere

theta, phi = np.mgrid[0.0:0.5*np.pi:n_theta*1j, 0.0:2.0*np.pi:n_phi*1j]

x = r*np.sin(theta)*np.cos(phi)
y = r*np.sin(theta)*np.sin(phi)
z = r*np.cos(theta)

# mimic the input array
# array columns phi, theta, value
# first n_theta entries: phi=0, second n_theta entries: phi=0.0315..
inp = []
for j in phi[0,:]:
    for i in theta[:,0]:
        r = i*2.0/np.pi
        l, m = xy_to_idx(*polar_to_xy(r, j), center, radius)
        try: 
            val = img2[l,m][0]
        except IndexError: 
            val = 255
        inp.append([j, i, val])
inp = np.array(inp)


#reshape the input array to the shape of the x,y,z arrays. 
c = inp[:,2].reshape((n_phi,n_theta)).T


#Set colours and render
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
#use facecolors argument, provide array of same shape as z
# cm.<cmapname>() allows to get rgba color from array.
# array must be normalized between 0 and 1
ax.plot_surface(
    x,y,z,  rstride=1, cstride=1, facecolors=cm.gray(c/c.max()), alpha=0.9, linewidth=1) 
ax.set_xlim([-1.1,1.1])
ax.set_ylim([-1.1,1.1])
ax.set_zlim([0,2.2])
ax.set_aspect("equal")
#ax.plot_wireframe(x, y, z, color="k") #not needed?!
plt.show()

In [ ]:
# Project points from sphere surface onto flat surface above. 

ww = rr*np.pi / 2.0
proj_r = d*np.tan(ww).flatten()
proj_x = proj_r*np.sin(th.flatten())
proj_y = proj_r*np.cos(th.flatten())
proj_pixel = img2[:,:,0].flatten()
proj_indices = np.array(np.where(ii*0 == 0)).T
crop = np.where((np.abs(proj_x)>width) | (np.abs(proj_y)>height))
proj_x = np.delete(proj_x, crop)
proj_y = np.delete(proj_y, crop)
proj_r = np.delete(proj_r, crop)
proj_pixel = np.delete(proj_pixel, crop)
proj_indices = np.delete(proj_indices, crop)

# Scatter projected pixels (slow, ugly)
plt.figure(figsize=(20,10))
plt.scatter(proj_x,-proj_y, c=proj_pixel, cmap=cm.gray, marker='s', edgecolors='none')
from matplotlib.patches import Rectangle
someX, someY = 1, 1
currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((np.min(xx), np.min(yy)), np.max(xx)-np.min(xx), np.max(yy)-np.min(yy),alpha=1,facecolor='none',edgecolor='red'))
plt.show()

# Interploate projected pixels to image grid
from scipy.interpolate import griddata
xvals = xx.flatten()
yvals = yy.flatten()
imgvals = griddata((proj_x,proj_y), proj_pixel, (xx, yy),method='nearest')
plt.figure(figsize=(20,10))
plt.imshow(imgvals,cmap=cm.gray)

In [ ]:
if UNDISTORT_DATASET:
  # use the griddata function to quickly create mapping using the above projection method
  map_indices = tuple(griddata((proj_x,proj_y), proj_indices, (xx, yy),method='nearest').reshape((-1,2)).T)

In [ ]:
if UNDISTORT_DATASET:
  dir_ = './distorted/'
  dir_out = './undistorted/'
  if not os.path.exists(dir_out): os.makedirs(dir_out)
  img_names = os.listdir(dir_)
  for name in img_names:
     distorted = skimage.io.imread(dir_+name)[:,:,0]
     undistorted = distorted[map_indices].reshape(xx.shape)
     skimage.io.imsave(dir_out+name, undistorted)

# Vertical Lines

In [ ]:
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny

In [ ]:
plt.figure("Lines", figsize=(20,10))
plt.clf()
dir_ = './undistorted_wald/'
name = 'frame0001.jpg'
img = skimage.io.imread(dir_+name)
plt.imshow(img,cmap='gray' )

h, theta, d = hough_line(img)
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
    y1 = (dist - img.shape[1] * np.cos(angle)) / np.sin(angle)
    plt.plot((0, img.shape[1]), (y0, y1), '-r')
plt.xlim((0, img.shape[1]))
plt.ylim((img.shape[0], 0))

In [ ]:
dir_ = './undistorted_wald/'
dir_out = './lines/'
if not os.path.exists(dir_out): os.makedirs(dir_out)
#img_names = sorted(os.listdir(dir_))
img_names = os.listdir(dir_)
for name in img_names:
    img = skimage.io.imread(dir_+name)
    # Canny Edges
    fig = plt.figure("Canny edges", frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
    plt.clf()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    edges = canny(img, 2, 1, 25)
    plt.imshow(edges, cmap='gray')
    # Probabilistic Lines
    fig = plt.figure("Probabilistic Lines",frameon=False,
                     figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
    plt.clf()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.imshow(img, cmap='gray')
    lines = probabilistic_hough_line(edges, threshold=10, line_length=10, line_gap=3)
    vertical_lines = []
    for line in lines:
        p0, p1 = line
        p1 = np.array(p1)
        p0 = np.array(p0)
        vec = p1 - p0
        if np.abs(vec[1]/(vec[0]+1e-1)) > 2:
            vertical_lines.append(line)
    for line in vertical_lines:
        p0, p1 = line
        plt.plot((p0[0], p1[0]), (p0[1], p1[1]), 'red')
    plt.xlim((0, img.shape[1]))
    plt.ylim((img.shape[0], 0))
    plt.savefig(dir_out+name)
    plt.show()

In [ ]:
dir_ = './undistorted_wald/'
dir_out = './lines/'
if not os.path.exists(dir_out): os.makedirs(dir_out)
#img_names = sorted(os.listdir(dir_))
img_names = os.listdir(dir_)
name = 'frame0444.jpg'
img = skimage.io.imread(dir_+name)
# Canny Edges
fig = plt.figure("Canny edges", frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
plt.clf()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
edges = canny(img, sigma=2, low_threshold=1, high_threshold=10)
plt.imshow(edges, cmap='gray')
# Probabilistic Lines
fig = plt.figure("Probabilistic Lines",frameon=False,
                 figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
plt.clf()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
plt.imshow(img, cmap='gray')
lines = probabilistic_hough_line(edges, threshold=10, line_length=10, line_gap=10)
vertical_lines = []
for line in lines:
    p0, p1 = line
    p1 = np.array(p1)
    p0 = np.array(p0)
    vec = p1 - p0
    if np.abs(vec[1]/(vec[0]+1e-1)) > 2:
        vertical_lines.append(line)
for line in vertical_lines:
    p0, p1 = line
    plt.plot((p0[0], p1[0]), (p0[1], p1[1]), 'red')
plt.xlim((0, img.shape[1]))
plt.ylim((img.shape[0], 0))
plt.savefig(dir_out+name)
plt.show()

In [ ]:
grad = np.copy(edges)
for n in range(5):
    fig = plt.figure("Gradients"+str(n), frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
    plt.clf()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.imshow(grad, cmap='gray')
    grad = vfilter(grad)
    plt.show()

In [ ]:
def vfilter(edges):
    indices = np.array([(i,j) for j in range(1,edges.shape[1]-1) for i in range(1,edges.shape[0]-1)])
    result = edges[:,:]
    for i,j in indices:
        result[i,j] = edges[i,j] and edges[i-1,j-1:j+2].any() and edges[i+1,j-1:j+2].any()
    return result
        

In [ ]:

up = np.diff(edges,axis=0) == 0
vert = (np.pad(up, ((1,0),(0,0)), 'constant', constant_values=0) + 
        np.pad(up, ((0,1),(0,0)), 'constant', constant_values=0)) * (edges.astype(int))
left = np.diff(edges,axis=1) == 0
hor =  (np.pad(left, ((0,0),(1,0)), 'constant', constant_values=0) + 
        np.pad(left, ((0,0),(0,1)), 'constant', constant_values=0)) * (edges.astype(int))

fig = plt.figure("Updiff", frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
plt.clf()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
plt.imshow(vert, cmap='gray')

fig = plt.figure("Leftdiff", frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
plt.clf()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
plt.imshow(hor, cmap='gray')

fig = plt.figure("vertonly", frameon=False, figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
plt.clf()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
vertonly = vert * (hor == 0)
plt.imshow(vertonly, cmap='gray')

In [ ]:
# Probabilistic Lines
fig = plt.figure("Probabilistic Lines",frameon=False,
                 figsize=(img.shape[1]/my_dpi, img.shape[0]/my_dpi), dpi=my_dpi)
plt.clf()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
plt.imshow(img, cmap='gray')
lines = probabilistic_hough_line(vertonly, threshold=20, line_length=50, line_gap=20)
vertical_lines = []
for line in lines:
    p0, p1 = line
    p1 = np.array(p1)
    p0 = np.array(p0)
    vec = p1 - p0
    if np.abs(vec[1]/(vec[0]+1e-1)) > 2:
        vertical_lines.append(line)
for line in vertical_lines:
    p0, p1 = line
    plt.plot((p0[0], p1[0]), (p0[1], p1[1]), 'red')
plt.xlim((0, img.shape[1]))
plt.ylim((img.shape[0], 0))
plt.show()